# 1. Download SAR from Sentinel 1

#### 1.1 SENTINEL-1 ***L3*** OCEAN WAVE SPECTRA

**Product ID**: WAVE_GLO_PHY_SWH_L2_NRT_014_016 (Near Real Time)
**Product ID**: WAVE_GLO_PHY_SWH_L2_MY_014_006 (Multi-Year Reprocessed)


*https://data.marine.copernicus.eu/product/WAVE_GLO_PHY_SPC-FWK_L3_NRT_014_002/files?path=WAVE_GLO_PHY_SPC-FWK_L3_NRT_014_002%2Fcmems-obs-wave_glo_phy_spc-fwk_nrt_s1a_l3_PT15S_202411%2F&subdataset=cmems-obs-wave_glo_phy_spc-fwk_nrt_s1a_l3_PT15S_202411*


**Variables**

Acess group obs_params at netcdf files.

- wave_spec (2D spectra [m^4])
- wavenumber_spec
- direction_spec
- time, longitude, latitude
- L2_partition_quality_flag (0=best)


First its necessary to have a copernicus user and password and install the library:

*pip install copernicusmarine*

In [12]:
import copernicusmarine

# Define dataset ID 
dataset_id = "cmems-obs-wave_glo_phy_spc-fwk_nrt_s1a_l3_PT15S"

output_dir = "./dados_copernicus/"

# # Download all available data
# result = copernicusmarine.get(
#     dataset_id=dataset_id,
#     output_directory=output_dir
# )
result = copernicusmarine.get(dataset_id=dataset_id, filter="*20201101*.nc")

print("Arquivos baixados:", result)


/Users/jtakeo/googleDrive/myProjects/wasp/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO - 2025-12-22T23:12:21Z - Selected dataset version: "202411"
INFO - 2025-12-22T23:12:21Z - Selected dataset part: "default"
WARNING - 2025-12-22T23:12:21Z - You are using the dataset cmems-obs-wave_glo_phy_spc-fwk_nrt_s1a_l3_PT15S, version '202411', part 'default'. This exact version and part of the dataset will be retired on the 2026-01-05T00:00:00.000Z. For more information you can check: https://marine.copernicus.eu/user-corner/product-roadmap/transition-information
INFO - 2025-12-22T23:12:22Z - Listing files on remote server...
21it [00:07,  2.86it/s]

Arquivos baixados: files=[FileGet(s3_url='s3://mdl-native-05/native/WAVE_GLO_PHY_SPC-FWK_L3_NRT_014_002/cmems-obs-wave_glo_phy_spc-fwk_nrt_s1a_l3_PT15S_202411/2020/10/dataset-wav-sar-l3-spc-nrt-global-s1a_20201012T0600Z_20201101T1500Z_P20201102T0402Z_12-OCT-2020-00H_lon_-052_lat_-56-rep.nc', https_url='https://s3.waw3-1.cloudferro.com/mdl-native-05/native/WAVE_GLO_PHY_SPC-FWK_L3_NRT_014_002/cmems-obs-wave_glo_phy_spc-fwk_nrt_s1a_l3_PT15S_202411/2020/10/dataset-wav-sar-l3-spc-nrt-global-s1a_20201012T0600Z_20201101T1500Z_P20201102T0402Z_12-OCT-2020-00H_lon_-052_lat_-56-rep.nc', file_size=14.909124374389648, last_modified_datetime='2024-10-14T11:01:15.664000+00:00', etag='"b6816e14817b52f642a23043bc46f581"', file_format='.nc', output_directory=PosixPath('.'), filename='dataset-wav-sar-l3-spc-nrt-global-s1a_20201012T0600Z_20201101T1500Z_P20201102T0402Z_12-OCT-2020-00H_lon_-052_lat_-56-rep.nc', file_path=PosixPath('WAVE_GLO_PHY_SPC-FWK_L3_NRT_014_002/cmems-obs-wave_glo_phy_spc-fwk_nrt_s1a_l

#### 1.2 SENTINEL-1 ***L2*** OCEAN WAVE SPECTRA



In [13]:
from datetime import date
import os, requests 
import warnings
import pandas as pd
import geopandas as gpd 
from shapely.geometry import shape 
from urllib3.exceptions import InsecureRequestWarning


warnings.filterwarnings('ignore', category=InsecureRequestWarning)


copernicus_user = "xxxx"   # Replace with your Copernicus username
copernicus_password = "xxxx" # Replace with your Copernicus password

# # WKT Representation for global coverage
# ft = "POLYGON((-180 -90, -180 90, 180 90, 180 -90, -180 -90))"

# WKT Representation for South Atlantic region
ft = "POLYGON((-60 -60, -60 10, 20 10, 20 -60, -60 -60))"

data_collection = "SENTINEL-1"


product_type = "OCN"  # Ocean products
acquisition_mode = "WV"  # Wave mode
polarization = "VV"  # Vertical-Vertical polarization


start_date = date(2020, 1, 1)  
end_date = date(2020, 1, 2)  

start_date_string = start_date.strftime("%Y-%m-%d")
end_date_string = end_date.strftime("%Y-%m-%d")

In [14]:

def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,  # Use the parameter instead of hardcoded value
        "password": password,  # Fix syntax error (remove comma)
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
        )
    return r.json()["access_token"]


def get_products(data_collection, product_type, acquisition_mode, start_date_string, end_date_string, footprint, token=None):
    all_products = []
    skip = 0
    top = 100  # Fetch in smaller batches
    
    while True:
        url = (f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' "
               f"and OData.CSC.Intersects(area=geography'SRID=4326;{footprint}') "
               f"and contains(Name, '{product_type}') "
               f"and contains(Name, '{acquisition_mode}') "
               f"and ContentDate/Start gt {start_date_string}T00:00:00.000Z "
               f"and ContentDate/Start lt {end_date_string}T00:00:00.000Z"
               f"&$skip={skip}&$top={top}&$count=True")
        
        # Make the request
        response = requests.get(url)
        json_data = response.json()
        
        batch = json_data.get("value", [])
        if not batch:
            break
            
        all_products.extend(batch)
        
        # If we got fewer results than requested, we've reached the end
        if len(batch) < top:
            break
            
        skip += top
    
    return all_products


all_products = get_products(data_collection, product_type, acquisition_mode, 
                           start_date_string, end_date_string, ft)
p = pd.DataFrame.from_dict(all_products)

# After getting the data, add a print statement to see what's available
print(f"Total products before filtering: {len(p)}")

if p.shape[0] > 0: # If we get data back
    p["geometry"] = p["GeoFootprint"].apply(shape)
    # Convert pandas dataframe to Geopandas dataframe by setting up geometry
    productDF = gpd.GeoDataFrame(p).set_geometry("geometry")
    
    # Print the first few product names to check what's available
    if len(productDF) > 0:
        print("Sample product names:")
        for name in productDF["Name"].head(5):
            print(f"  - {name}")
    
    # Filter for VV polarization (look for 2SSV pattern in the name)
    productDF = productDF[productDF["Name"].str.contains("OCN") & 
                          productDF["Name"].str.contains("WV")]
    
    print(f"Total Wave mode products found: {len(productDF)}")
    productDF["identifier"] = productDF["Name"].str.split(".").str[0]
    allfeat = len(productDF)
    
    if allfeat == 0: # If L2A tiles are not available in current query
        print(f"No tiles found for date range {start_date_string} to {end_date_string}")
    else: # If L2A tiles are available in current query
        # Create directory before the loop
        output_dir = "sentinel_downloads"  # Change this to your preferred path
        os.makedirs(output_dir, exist_ok=True)
        
        # download all tiles from server
        for index,feat in enumerate(productDF.iterfeatures()):
            try:
                # Create requests session 
                session = requests.Session()
                # Get access token based on username and password
                keycloak_token = get_keycloak(copernicus_user,copernicus_password)
                
                session.headers.update({"Authorization": f"Bearer {keycloak_token}"})
                url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({feat['properties']['Id']})/$value"
                response = session.get(url, allow_redirects=False)
                while response.status_code in (301, 302, 303, 307):
                    url = response.headers["Location"]
                    response = session.get(url, allow_redirects=False)
                print(feat["properties"]["Id"])
                file = session.get(url, verify=False, allow_redirects=True)

                with open(
                    f"{output_dir}/{feat['properties']['identifier']}.zip", # Update the path in the loop
                    "wb",
                ) as p:
                    print(feat["properties"]["Name"])
                    p.write(file.content)
            except Exception as e:
                print(f"Error processing item {feat['properties']['Id']}: {str(e)}")
else : # If no tiles found for given date range and AOI
    print('no data found')




Total products before filtering: 10
Sample product names:
  - S1A_WV_OCN__2SSV_20200101T032506_20200101T033017_030601_03817A_FEF0.SAFE
  - S1A_WV_OCN__2SSV_20200101T075924_20200101T080800_030604_038193_4EAE.SAFE
  - S1A_WV_OCN__2SSV_20200101T170431_20200101T171109_030609_0381C5_114A.SAFE
  - S1A_WV_OCN__2SSV_20200101T044245_20200101T044854_030602_038182_A2EB.SAFE
  - S1B_WV_OCN__2SSV_20200101T041014_20200101T042018_019618_02513E_2510.SAFE
Total Wave mode products found: 10
Error processing item 302080f0-c4b0-596b-8765-a90781e64cf2: Keycloak token creation failed. Reponse from the server was: {'error': 'invalid_grant', 'error_description': 'Invalid user credentials'}
Error processing item 8424d333-ed02-52c5-8168-51d05f2bd242: Keycloak token creation failed. Reponse from the server was: {'error': 'invalid_grant', 'error_description': 'Invalid user credentials'}
Error processing item a62fc2b6-4716-5d44-a7ad-960ef9720464: Keycloak token creation failed. Reponse from the server was: {'error